In [ ]:
!pip install -U sentence-transformers

In [ ]:
from mongoDB_API import *
from tqdm import tqdm
import pandas as pd
import torch
from sentence_transformers import SentenceTransformer, util
from google.colab import files
import glob
import gc

model = SentenceTransformer('all-MiniLM-L6-v2')
client = MongoDBClient(IP = "34.142.18.57")


In [ ]:
const = 10**6
done = [int(i.split('_')[1].split('.')[0]) for i in glob.glob("Embeddings/*")]
start = 1
indices = [i for i in range(start, 24) if i not in done]

for temp_i in tqdm(indices):
  i = temp_i * const + 1
  print(f"\nProcessing Documents: {i} - {i+const-1}")
  cursor = client.get_data("paper", {}, [], i, const - 1)
  df = pd.DataFrame(list(cursor))
  del(cursor)

  df = (df.title + " " + df.abstract).astype(str)
  embeddings = model.encode(df, show_progress_bar=True, convert_to_tensor=True)
  print(f'Embeddings/tensor_{temp_i}.pt')
  del(df)

  gc.collect()
  
  torch.save(embeddings, f'Embeddings/tensor_{temp_i}.pt')
  del(embeddings)
  torch.cuda.empty_cache()


In [ ]:
pip install annoy

In [ ]:
from annoy import AnnoyIndex
import random
import numpy as np
from tqdm.notebook import tqdm
import torch

f = 384

In [ ]:
t = AnnoyIndex(f, 'angular')  # Length of item vector that will be indexed
t.set_seed(43)
t.on_disk_build('searcher.ann')


In [ ]:
index = 0
for i in tqdm(range(24)):
  temp = torch.load(f"/home/stylianosc/folder/Embeddings/tensor_{i}.pt", map_location=torch.device('cpu'))
  for j in tqdm(range(temp.shape[0]), leave = False):
    temp_doc = temp[j]
    t.add_item(index, temp_doc)
    
    index += 1

In [ ]:
t.get_n_items()

2000000

In [ ]:
t.build(200)

# ...



In [ ]:
u = AnnoyIndex(f, 'angular')
u.load('searcher.ann') # super fast, will just mmap the file
#print(u.get_nns_by_item(0, 100)) # will find the 1000 nearest neighbors

In [ ]:
u.get_n_items()

0